<a href="https://colab.research.google.com/github/NancyWangareh/HEART-DISEASE/blob/main/RAINFALL_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

In [ ]:
c("/content/Rainfall(1).csv")
df.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1,1025.9,19.9,18.3,16.8,13.1,72,49,yes,9.3,80.0,26.3
1,2,1022.0,21.7,18.9,17.2,15.6,81,83,yes,0.6,50.0,15.3
2,3,1019.7,20.3,19.3,18.0,18.4,95,91,yes,0.0,40.0,14.2
3,4,1018.9,22.3,20.6,19.1,18.8,90,88,yes,1.0,50.0,16.9
4,5,1015.9,21.3,20.7,20.2,19.9,95,81,yes,0.0,40.0,13.7


In [ ]:
df.shape

(366, 12)

In [ ]:
df.dtypes

,0
day,int64
pressure,float64
maxtemp,float64
temparature,float64
mintemp,float64
dewpoint,float64
humidity,int64
cloud,int64
rainfall,object
sunshine,float64


In [ ]:
df.isnull().sum

<bound method DataFrame.sum of        day  pressure   maxtemp  temparature  mintemp  dewpoint  humidity   \
0    False      False    False        False    False     False      False   
1    False      False    False        False    False     False      False   
2    False      False    False        False    False     False      False   
3    False      False    False        False    False     False      False   
4    False      False    False        False    False     False      False   
..     ...        ...      ...          ...      ...       ...        ...   
361  False      False    False        False    False     False      False   
362  False      False    False        False    False     False      False   
363  False      False    False        False    False     False      False   
364  False      False    False        False    False     False      False   
365  False      False    False        False    False     False      False   

     cloud   rainfall  sunshine           winddirection  windspeed  
0     False     False     False                   False      False  
1     False     False     False                   False      False  
2     False     False     False                   False      False  
3     False     False     False                   False      False  
4     False     False     False                   False      False  
..      ...       ...       ...                     ...        ...  
361   False     False     False                   False      False  
362   False     False     False                   False      False  
363   False     False     False                   False      False  
364   False     False     False                   False      False  
365   False     False     False                   False      False  

[366 rows x 12 columns]>

In [ ]:
df.info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   day                     366 non-null    int64  
 1   pressure                366 non-null    float64
 2   maxtemp                 366 non-null    float64
 3   temparature             366 non-null    float64
 4   mintemp                 366 non-null    float64
 5   dewpoint                366 non-null    float64
 6   humidity                366 non-null    int64  
 7   cloud                   366 non-null    int64  
 8   rainfall                366 non-null    object 
 9   sunshine                366 non-null    float64
 10           winddirection  365 non-null    float64
 11  windspeed               365 non-null    float64
dtypes: float64(8), int64(3), object(1)
memory usage: 34.4+ KB


In [ ]:
df.columns

Index(['day', 'pressure ', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'humidity ', 'cloud ', 'rainfall', 'sunshine', '         winddirection',
       'windspeed'],
      dtype='object')

In [ ]:
df.tail(5)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
361,27,1022.7,18.8,17.7,16.9,15.0,84,90,yes,0.0,30.0,18.4
362,28,1026.6,18.6,17.3,16.3,12.8,75,85,yes,1.0,20.0,25.9
363,29,1025.9,18.9,17.7,16.4,13.3,75,78,yes,4.6,70.0,33.4
364,30,1025.3,19.2,17.3,15.2,13.3,78,86,yes,1.2,20.0,20.9
365,31,1026.4,20.5,17.8,15.5,13.0,74,66,no,5.7,20.0,23.3


In [ ]:
df.describe

<bound method NDFrame.describe of      day  pressure   maxtemp  temparature  mintemp  dewpoint  humidity   \
0      1     1025.9     19.9         18.3     16.8      13.1         72   
1      2     1022.0     21.7         18.9     17.2      15.6         81   
2      3     1019.7     20.3         19.3     18.0      18.4         95   
3      4     1018.9     22.3         20.6     19.1      18.8         90   
4      5     1015.9     21.3         20.7     20.2      19.9         95   
..   ...        ...      ...          ...      ...       ...        ...   
361   27     1022.7     18.8         17.7     16.9      15.0         84   
362   28     1026.6     18.6         17.3     16.3      12.8         75   
363   29     1025.9     18.9         17.7     16.4      13.3         75   
364   30     1025.3     19.2         17.3     15.2      13.3         78   
365   31     1026.4     20.5         17.8     15.5      13.0         74   

     cloud  rainfall  sunshine           winddirection  windspeed  
0        49      yes       9.3                    80.0       26.3  
1        83      yes       0.6                    50.0       15.3  
2        91      yes       0.0                    40.0       14.2  
3        88      yes       1.0                    50.0       16.9  
4        81      yes       0.0                    40.0       13.7  
..      ...      ...       ...                     ...        ...  
361      90      yes       0.0                    30.0       18.4  
362      85      yes       1.0                    20.0       25.9  
363      78      yes       4.6                    70.0       33.4  
364      86      yes       1.2                    20.0       20.9  
365      66       no       5.7                    20.0       23.3  

[366 rows x 12 columns]>